In [ ]:
# Just the normal prequisites for using matplotlib and numpy in a Jupyter notebook
%matplotlib inline
# Use the svg backend, in my opinion it just makes better looking plots
%config InlineBackend.figure_format = 'svg'

import os
from scipy import signal
import scipy.fftpack
from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import Circuit, SubCircuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Doc.ExampleTools import find_libraries
from PySpice.Math import *
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import unittest

import SchemDraw as schemdraw
import SchemDraw.elements as elm

# The spice and kicad modules are located above us
import sys
sys.path.append('../../lib/python')

import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

directory_path = Path(os.path.abspath('')).resolve()


![module](https://img.shields.io/badge/module-vca-yellow)
![status](https://img.shields.io/badge/status-work%20in%20progress-orange)

## *pegel*

<a href="https://photos.app.goo.gl/pg6iZUC32rTKX5LC7"><img src="https://spielhuus.github.io/elektrophon/images/kontrast-logo-tmb.jpg" height="300px" align="right"></a>

***table of contents***

* [*about*](#about)
* [*construction*](#construction)
* [*calibration*](#calibration)
* [*usage*](#usage)
* [*credits*](#credits)
* [*links*](#links)
* [*changelog*](#changelog)
  
<br/><br/><br/><br/>

## *about*

pegel is a voltage controlled amplifier (vca). there are various designs to implement a vca. rod elliott (ESP) has an article on vca techniques [[1][1]]. diy synthesizer modules are usualy designed with an integrated vca chip [[2][2]]  or with a differential amplifier [[3][3]]. the integrated vca chips are either obsolete or rather expensive. the differential amplifier can be built with transistors only. the design has some downsides. even in the simulation the result is not symetryc and has a dc offset from the control voltage. this needs a lot of trimming to get an accurate result. but the biggest downside is, that this design can not do proper amplitude modulation (am). when the carrier signal goes below zero the base signal is completely muted. 

In [ ]:
#load the diffpair schema
kicad_netlist_path = directory_path.joinpath('diffpair', 'diffpair.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

In [ ]:
#simulate diffpair with envelope
diffpair_envelope = parser.build_circuit(ground=5)
diffpair_envelope.include(spice_library['BC547B'])
diffpair_envelope.V('1', '+15V', diffpair_envelope.gnd, 'DC 15')
diffpair_envelope.V('2', '-15V', diffpair_envelope.gnd, 'DC -15')
diffpair_envelope.V('3', 'Vin_a', diffpair_envelope.gnd, 'DC 0V AC 0V SIN(0 25m 1k)')
diffpair_envelope.V('4', 'Vin_b', diffpair_envelope.gnd, 'DC 0V AC 0V PULSE(200m -4.5 1m 1u 15m 1u)')
simulator = diffpair_envelope.simulator(temperature=25, nominal_temperature=25)
analysis_envelope = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
#simulate  amplitude modulation
diffpair_am = parser.build_circuit(ground=5)
diffpair_am.include(spice_library['BC547B'])
diffpair_am.V('1', '+15V', diffpair_am.gnd, 'DC 15')
diffpair_am.V('2', '-15V', diffpair_am.gnd, 'DC -15')
diffpair_am.V('3', 'Vin_a', diffpair_am.gnd, 'DC 0V AC 0V SIN(0 25m 1k)')
diffpair_am.V('4', 'Vin_b', diffpair_am.gnd, 'DC 0V AC 0V SIN(0 5 100)')

simulator = diffpair_am.simulator(temperature=25, nominal_temperature=25)
analysis_am = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
#plot the results
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

#plot the image
img_diffpair_path = directory_path.joinpath('diffpair', 'diffpair.png')
img_diffpair = mpimg.imread(str(img_diffpair_path))
im = ax0.imshow(img_diffpair)
ax0.axis('off')
ax0.set_title('long tailed pair differential amplifier', y=-0.2)
        
#plot with envelope
ax1.plot(analysis_envelope['Vin_b'].abscissa*1000, analysis_envelope['Vin_b'] * 0.2 * -1, c='grey')  # envelope input (scaled)
ax1.plot(analysis_envelope['Vout_b'].abscissa*1000, analysis_envelope['Vout_b'] - analysis_envelope['Vout_a'], c='orange')  # differential output
ax1.grid()
ax1.set_xlabel('t [ms]')
ax1.set_ylabel('[V]')
ax1.set_title('output with a cv envelope', y=-0.2)

#plot the amplitude modulation
ax2.plot(analysis_am['Vout_b'].abscissa*1000, analysis_am['Vout_b'] - analysis_am['Vout_a'], c='orange')  # differential output
ax2.plot(analysis_am['Vout_b'].abscissa*1000, analysis_am['Vin_b'] * 0.2 * -1, c='grey')  # differential output
ax2.grid()
ax2.set_xlabel('t [ms]')
ax2.set_ylabel('[V]')
ax2.set_title('output with a sine control voltage.', y=-0.2)
plt.show()


more promising is it to use a four quadrant multiplier, also known as gilbert cell. the gilbert cell is designed around two differential amplifiers. the carrier signal switches between those. the result is a multiplication of the input voltages. the gilbert cell is mostly used for amplitude modulation in radio transmission. if you look at the integratd circuits like the 633 it can modulate signals up in the gigahertz range. the gilbert cell is not described in all details here, there is a good introduction from w2aew [[4][4]].




In [ ]:
#load the gilbert_cell schema
kicad_netlist_path = directory_path.joinpath('gilbert_cell', 'gilbert_cell.cir')
parser = SpiceParser(path=str(kicad_netlist_path))

In [ ]:
#simulate with envelope
gilbert_cell_envelope = parser.build_circuit(ground=5)
gilbert_cell_envelope.include(spice_library['BC547B'])
gilbert_cell_envelope.include(spice_library['TL072'])

gilbert_cell_envelope.V('1', '+15V', gilbert_cell_envelope.gnd, 'DC 15')
gilbert_cell_envelope.V('2', '-15V', gilbert_cell_envelope.gnd, 'DC -15')
gilbert_cell_envelope.V('3', 'NC_01', gilbert_cell_envelope.gnd, 'DC 0V AC 0V SIN(0 100m 1k)')
gilbert_cell_envelope.V('4', 'CV_IN', gilbert_cell_envelope.gnd, 'DC 0V AC 0V PULSE(0 100m 1m 0u 10m 1u)')

simulator = gilbert_cell_envelope.simulator(temperature=25, nominal_temperature=25)
analysis_gilbert_cell_envelope = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
#simulate ring modulation
gilbert_am = parser.build_circuit(ground=5)
gilbert_am.include(spice_library['BC547B'])
gilbert_am.include(spice_library['TL072'])

gilbert_am.V('1', '+15V', gilbert_am.gnd, 'DC 15')
gilbert_am.V('2', '-15V', gilbert_am.gnd, 'DC -15')
gilbert_am.V('3', 'NC_01', gilbert_am.gnd, 'DC 0V AC 0V SIN(0 100m 1k)')
gilbert_am.V('4', 'CV_IN', gilbert_am.gnd, 'DC 0V AC 0V SIN(0 100m 100)')

simulator = gilbert_am.simulator(temperature=25, nominal_temperature=25)
analysis_gilbert_cell_am = simulator.transient(step_time=1@u_us, end_time=10@u_ms)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

#plot the image
img_diffpair_path = directory_path.joinpath('gilbert_cell', 'g2074.png')
img_gilbert = mpimg.imread(str(img_diffpair_path))
im = ax0.imshow(img_gilbert)
ax0.axis('off')
ax0.set_title('gilbert cell.', y=-0.2)

#plot the envelope
ax1.plot(analysis_gilbert_cell_envelope['CV_IN'].abscissa*1000, analysis_gilbert_cell_envelope['CV_IN'] / 7.1, c='grey')  # envelope
ax1.plot(analysis_gilbert_cell_envelope['OUT_A'].abscissa*1000, analysis_gilbert_cell_envelope['OUT_A'] - analysis_gilbert_cell_envelope['OUT_B'], c='orange')  # differential output
ax1.grid()
ax1.set_xlabel('t [ms]')
ax1.set_ylabel('[V]')
ax1.set_title('output with an envelope control voltage.', y=-0.2)

#plot the ring modulation
ax2.plot(analysis_gilbert_cell_am['CV_IN'].abscissa*1000, analysis_gilbert_cell_am['CV_IN'] / 7.1, c='grey')  # modulating
ax2.plot(analysis_gilbert_cell_am['OUT_A'].abscissa*1000, analysis_gilbert_cell_am['OUT_A'] - analysis_gilbert_cell_am['OUT_B'], c='orange')  # differential output
ax2.legend(('Vinput b  [V]', 'Vinput a [V]', 'Vout [V]'), loc=(.8, .8))
ax2.grid()
ax2.set_xlabel('t [ms]')
ax2.set_ylabel('[V]')
ax2.set_title('output with a sine control voltage.', y=-0.2)

plt.show()


this is not real amplitude modulation. when the signal is negative the phase of the output is inverted. this can be adjusted by the bias voltage of the control voltage. the control voltage has to be positive at all time. 


In [ ]:
#plot the amplitude modulation
gilbert_real_am = parser.build_circuit(ground=5)
gilbert_real_am.include(spice_library['BC547B'])

gilbert_real_am.V('1', '+15V', gilbert_am.gnd, 'DC 15')
gilbert_real_am.V('2', '-15V', gilbert_am.gnd, 'DC -15')
gilbert_real_am.V('3', 'NC_01', gilbert_am.gnd, 'DC 0V AC 0V SIN(0 50m 1k)')
gilbert_real_am.V('4', 'CV_IN', gilbert_am.gnd, 'DC 0V AC 0V SIN(50m 50m 100)')

simulator = gilbert_real_am.simulator(temperature=25, nominal_temperature=25)
analysis_gilbert_real_am = simulator.transient(step_time=1@u_us, end_time=20@u_ms)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

#plot real amplitude modulation
ax0.plot(analysis_gilbert_real_am['NC_01'].abscissa*1000, analysis_gilbert_real_am['NC_01'] / 7.1)  # differential output
ax0.plot(analysis_gilbert_real_am['CV_IN'].abscissa*1000, (analysis_gilbert_real_am['CV_IN'] -0.05@u_V)/7.1)  # differential output
ax0.plot(analysis_gilbert_real_am['OUT_A'].abscissa*1000, analysis_gilbert_real_am['OUT_A'] - analysis_gilbert_real_am['OUT_B'])  # differential output
ax0.grid()
ax0.set_xlabel('t [s]')
ax0.set_ylabel('[V]')
ax0.set_title('output with a sine control voltage, biased for am (Vin_b is scaled).', y=-0.2)

y = analysis_gilbert_real_am['OUT_A'] - analysis_gilbert_real_am['OUT_B']
N = y.size
# sample spacing
T = 1.0 / 10000.0
x = np.linspace(0.0, N*T, N)
yf = scipy.fftpack.fft(y)
xf = np.linspace(0.0, 1.0//(2.0*T), N//2)
ax1.plot(xf, 2.0/N * np.abs(yf[:N//2]))

#am bode diagram
x = np.linspace(0.0, N*T, N)
y = analysis_gilbert_real_am['OUT_A'] - analysis_gilbert_real_am['OUT_B']
yf = scipy.fftpack.fft(y)
xf = np.linspace(0.0, 1.0//(2.0*T), N//2)
ax2.plot(yf)

#dsb-sc bode diagram
#y = analysis_gilbert_am['OUT_A'] - analysis_gilbert_am['OUT_B']
#yf = scipy.fftpack.fft(y)
#xf = np.linspace(0.0, 1.0//(2.0*T), N)
#ax2.plot(xf, 2.0/N * np.abs(yf[:N//2]))

plt.show()


## *construction*

for the final circuit input and output buffering and biasing is needed. the buffering is done with opamps. 


In [ ]:
directory_path = Path(os.path.abspath('')).resolve()
kicad_netlist_path = directory_path.joinpath('main', 'main.cir')
parser = SpiceParser(path=str(kicad_netlist_path))
ad633_schema = parser.build_circuit(ground=5)
ad633_schema.include(spice_library['AD633'])
ad633_schema.include(spice_library['TL072'])

#ad633_schema.V('1', '+15V', ad633_schema.gnd, 'DC 15V')
#ad633_schema.V('2', '-15V', ad633_schema.gnd, 'DC -15V')
#ad633_schema.V('3', 'X1', ad633_schema.gnd, 'DC 0V AC 10V SINE(0 1 440)')
#ad633_schema.V('4', 'Y1', ad633_schema.gnd, 'DC 0V AC 10V SINE(0 5 100)')
#ad633_schema.V('4', 'Y1', ad633_schema.gnd, 'DC 0V AC 10V PULSE(0 -10 0 0 500m 100u 600m)')
#ad633_schema.V('5', 'Z', ad633_schema.gnd, 'DC 0V AC 0V')

print(str(ad633_schema))


In [ ]:
import wave
import struct

SAMPLING_RATE = 44100

def lin_interp(x0, x1, y0, y1, x):
    x0 = float(x0)
    x1 = float(x1)
    y0 = float(y0)
    y1 = float(y1)
    x = float(x)
    return y0 + (x - x0) * (y1 - y0) / (x1 - x0)

def write_wav(times, voltages, filename, clipping):
	with wave.open(filename, 'w') as w:
		w.setparams((1, 2, SAMPLING_RATE, 0, 'NONE', 'not compressed'))
		m = max(max(voltages), -min(voltages))
		vrange = clipping if clipping else m

		values = bytes([])
		t = 0.0
		step = 1.0 / SAMPLING_RATE

		for i in range(len(voltages)-1):
			while times[i] <= t * step < times[i+1]:
				sample = lin_interp(times[i], times[i+1],
					voltages[i], voltages[i+1], t*step) / vrange
				sample = 1 if sample >1 else sample
				sample = -1 if sample <-1 else sample
				d = struct.pack('<h',int(32767 * sample))
				values += d
				t += 1

		w.writeframes(values)


fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=False, figsize=(18, 6))

simulator = ad633_schema.simulator(temperature=25, nominal_temperature=25)
samplerate = 1 / 44100
ad633_analysis = simulator.transient(step_time=22.675@u_us, end_time=4@u_s)

#for d in ad633_analysis.NC_01 :
#    print( '%f %f' % (d.abscissa*1e6, float(d)) )

#np.array(ad633_analysis['NC_01']).astype('int16').tofile('testfile.raw')
#write("example.wav", 44100, np.array(ad633_analysis['NC_01']))
times = []
for a in ad633_analysis.NC_01.abscissa :
    times.append( float(a)) 

write_wav(times, ad633_analysis.NC_01.astype(np.float) , 'new_out.wav', 0)

ax0.plot(ad633_analysis.NC_01.abscissa, ad633_analysis.NC_01)

#ax0.plot(ad633_analysis['X1'])  # cv out
#ax0.plot(ad633_analysis['Y1'])  # cv out
#ax0.plot(ad633_analysis['NC_01'])  # cv out
ax0.legend(('Vout [V]'), loc=(.8, .8))
ax0.grid()
ax0.set_xlabel('t [s]')
ax0.set_ylabel('[V]')
ax0.set_title('output with a cv envelope (Vin_b is scaled).', y=-0.2)

plt.tight_layout()
plt.show()


In [ ]:
import IPython
IPython.display.Audio('new_out.wav')

In [ ]:
import unittest

class TestInputVoltages(unittest.TestCase):
    
    def test_audio_a(self):
        self.assertAlmostEqual(4.0, np.average(np.array(input_analysis.audio_a)), places=3, msg='test the audio 1 in average')
        self.assertAlmostEqual(0.036, np.max(np.array(input_analysis.audio_a))-np.min(np.array(input_analysis.audio_a)), places=3, msg='test the audio 1 in power')

    def test_audio_b(self):
        self.assertAlmostEqual(0.0, np.average(np.array(input_analysis.audio_b)), places=2, msg='test the audio 2 in average')
        self.assertAlmostEqual(0.036, np.max(np.array(input_analysis.audio_b))-np.min(np.array(input_analysis.audio_b)), places=3, msg='test the audio 2 in power')

    def test_envelope(self):
        self.assertAlmostEqual(0.0, np.average(np.array(input_analysis.env)), places=2, msg='test the envelope in average')
        self.assertAlmostEqual(0.01, np.max(np.array(input_analysis.env))-np.min(np.array(input_analysis.env)), places=2, msg='test the envelope in power')

    def test_cv(self):
        self.assertAlmostEqual(2.0, np.average(np.array(input_analysis.cv)), places=2, msg='test the cv average')
        self.assertAlmostEqual(0.056, np.max(np.array(input_analysis.cv))-np.min(np.array(input_analysis.cv)), places=2, msg='test the cv power')

unittest.main(argv=[''], verbosity=2, exit=False)

## *references*

- [VCA Techniques Investigated][1] Rod Elliott (ESP)
- [Popular Electronics][2] Keyiing and VCA citcuits for electronic music instruments 
- [VCA-1][3] Thomas Henry CA3080 vca
- [VCA-3][4] René Schmitz differential pair vca
- [#223][5]: Basics of the Gilbert Cell | Analog Multiplier | Mixer | Modulator
- [#224][6]: AM & DSB-SC Modulation with the Gilbert Cell


[1]: https://sound-au.com/articles/vca-techniques.html
[2]: https://tinaja.com/glib/pop_elec/mus_keying_vca_1+2_75.pdf
[3]: https://www.birthofasynth.com/Thomas_Henry/Pages/VCA-1.html
[4]: https://www.schmitzbits.de/vca3.png
[5]: https://www.youtube.com/watch?v=7nmmb0pqTU0&t=2s
[6]: https://www.youtube.com/watch?v=38OQub2Vi2Q
[7]: http://www.ecircuitcenter.com/Circuits/BJT_Diffamp1/BJT_Diffamp1.htm
